This example solves the same AR(1) problem, but the purpose here is to demonstration the versatitily of `linkalman`. It has the following three changes:

1. three different methods to illustrate all three optimizing techniques
2. use customized ft with BaseOpt instead of BaseConstantModel
3. use nlopt as an alternative solver to illustrate flexibility of linkalman solver config

In [1]:
"""
AR(1) model
"""
import numpy as np
import pandas as pd
import linkalman
import scipy
from linkalman.models import BaseOpt as BM
from linkalman.core.utils import Constant_M
import nlopt
%matplotlib inline 


# Initialization
Instead of relying on `BaseConstantModel`, here I directly define the function `my_ft`. An `ft` should have two required positional arguments: `theta` and `T`. It may also contain keyword arguments. In defining a constant model, I also leverage the `linkalman.core.utils.Constant_M` module. If you have a long time series with mostly constant system dynamic matrices, you can use `Constant_M` to save storage spaces. 

In [2]:
def my_ft(theta, T, x_0=0):
    """
    AR(1) model with noise. In general, MLE is biased, so the focus should be 
    more on prediction fit, less on parameter estimation. The 
    formula here for Ar(1) is:
    y_t = c + Fy_{t-1} + epsilon_{t-1}
    """
    # Define theta
    phi_1 = 1 / (np.exp(theta[0])+1)
    sigma = np.exp(theta[1]) 
    sigma_R = np.exp(theta[3]) 
    # Generate F
    F = np.array([[phi_1]])
    # Generate Q
    Q = np.array([[sigma]]) 
    # Generate R
    R = np.array([[sigma_R]])
    # Generate H
    H = np.array([[1]])
    # Generate B
    B = np.array([[theta[2]]])
    # Generate D
    D = np.array([[0]])
    
    # Build Mt
    Ft = Constant_M(F, T)
    Bt = Constant_M(B, T)
    Qt = Constant_M(Q, T)
    Ht = Constant_M(H, T)
    Dt = Constant_M(D, T)
    Rt = Constant_M(R, T)
    xi_1_0 = theta[2] * x_0 / (1 - phi_1)  # calculate stationary mean, x_0 is already np.ndarray
    P_1_0 = np.array([[sigma /(1 - phi_1 * phi_1)]])  # calculate stationary cov
    
    Mt = {'Ft': Ft,
          'Bt': Bt,
          'Qt': Qt,
          'Ht': Ht,
          'Dt': Dt,
          'Rt': Rt,
          'xi_1_0': xi_1_0,
          'P_1_0': P_1_0}
    return Mt


# Solver

In [3]:
def my_solver(param, obj_func, **kwargs):
    """
    More complex solver function than the simple AR(1) case.
    The purpose is to provide an example of flexiblity of
    building solvers. Note I also suppress grad in nlopt_obj, 
    as linkalman uses only non-gradient optimizers
    """
    def nlopt_obj(x, grad, **kwargs):
        fval_opt = obj_func(x)
        if kwargs.get('verbose', False):
            print('fval: {}'.format(fval_opt))
        return fval_opt

    opt = nlopt.opt(nlopt.LN_BOBYQA, param.shape[0])
    obj = lambda x, grad: nlopt_obj(x, grad, **kwargs)
    opt.set_max_objective(obj)
    opt.set_xtol_rel(kwargs.get('xtol_rel', opt.get_xtol_rel()))
    opt.set_ftol_rel(kwargs.get('ftol_rel', opt.get_ftol_rel()))
    theta_opt = opt.optimize(param)
    fval_opt = opt.last_optimum_value()
    if kwargs.get('verbose_opt', False):
        print('fval: {}'.format(fval_opt))
        
    return theta_opt, fval_opt

In [4]:
# Initialize the model
x = 1
model = BM()
model.set_f(my_ft, x_0=x * np.ones([1, 1]))
model.set_solver(my_solver, xtol_rel=1e-4, verbose=True) 


# Generate Synthetic Data

In [5]:
# Some initial parameters
theta = np.array([0, -0.1, 0.1, 1])
T = 365
train_split_ratio = 0.7
forecast_cutoff_ratio = 0.8  

# Split train data
train_split_t = np.floor(T * train_split_ratio).astype(int)

# Generate missing data for forcasting
forecast_t = np.floor(T * forecast_cutoff_ratio).astype(int)

# If we want AR(1) with non-zero stationary mean, we should proivde a constant 
x_col = ['const']
Xt = pd.DataFrame({x_col[0]: x * np.ones(T)})  # use x to ensure constant model

# Build simulated data
df, y_col, xi_col = model.simulated_data(input_theta=theta, Xt=Xt)

# Store fully visible y for comparison later
df['y_0_vis'] = df.y_0.copy()  

# Splits models into three groups
is_train = df.index < train_split_t
is_test = (~is_train) & (df.index < forecast_t)
is_forecast = ~(is_train | is_test)

# Create a training and test data
df_train = df.loc[is_train].copy()

# Build two kinds of test data (full data vs. test data only)
df_test = df.copy()  

# Create an offset
df_test.loc[is_forecast, ['y_0']] = np.nan

# LLY
First, I use numerical methods. It is the preferred methods over EM algorithm, because EM need score function to be effective, which is rather limiting. `linkalman` makes a compromise to gain flexibility in handling missing measurements and customized `ft`.

In [6]:
# Fit data using LLY:
theta_init = np.random.rand(len(theta))
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, 
              method='LLY')
theta_LLY = model.theta_opt

fval: -577.4956255811364
fval: -573.8692505684113
fval: -577.7483144723885
fval: -597.6957969116199
fval: -574.4687302800978
fval: -581.9720539546967
fval: -596.3707246236846
fval: -571.6204274874208
fval: -581.0345733181947
fval: -566.3601180316119
fval: -561.5056871047798
fval: -560.5884455062071
fval: -560.4115693123397
fval: -560.3100617806385
fval: -559.954261045995
fval: -559.3702066752151
fval: -558.5636438437797
fval: -557.7357582353496
fval: -557.1551313832518
fval: -556.7487950123475
fval: -556.7351261479282
fval: -556.7227396577634
fval: -556.6961317138232
fval: -556.8266151808721
fval: -556.7158755145693
fval: -556.6344815269573
fval: -556.6115520481641
fval: -556.597979926823
fval: -556.5805644434367
fval: -556.5589956811486
fval: -556.5545626142995
fval: -556.5526255893946
fval: -556.6438511576076
fval: -556.5555916563309
fval: -556.5952720533854
fval: -556.5501642275244
fval: -556.5494787531394
fval: -556.5487044270972
fval: -556.5479719258241
fval: -556.5468997107449
fv

# Use EM to Cold Start Optimization
One may combine EM and LLY method. EM methods, with score functions have very fast convergence at the beginning. Interested readers may design their own solvers to compute the saddle point directly. 

In [7]:
# Fit data using both methods:
theta_init = np.random.rand(len(theta))
model.set_solver(my_solver, xtol_rel=1e-3, ftol_rel=1e-3, verbose_opt=True) 
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, method='EM', num_EM_iter=20)
model.set_solver(my_solver, xtol_rel=1e-4, verbose=True) 
model.fit(df_train, model.theta_opt, y_col=y_col, x_col=x_col, method='LLY')
theta_mix = model.theta_opt

fval: -718.0616510262953
fval: -717.7133759737154
fval: -727.1158683077156
fval: -727.0517019894729
fval: -726.9778661994509
fval: -726.8973457599078
fval: -726.7838870855956
fval: -726.8373133725111
fval: -727.156361584093
fval: -726.7995475850662
fval: -726.7993119039363
fval: -726.3722693008765
fval: -726.0565808864294
fval: -725.8286350406881
fval: -725.6040538418486
fval: -725.0685575431858
fval: -724.5443743315608
fval: -724.1179071946653
fval: -723.7530103914132
fval: -723.391365927951
fval: -557.2305061899697
fval: -557.9511729778741
fval: -557.2388866144707
fval: -559.8259573658924
fval: -596.0879062791444
fval: -561.210736438695
fval: -557.2621415150737
fval: -556.7942025031384
fval: -589.8312713241271
fval: -557.1333689188539
fval: -557.0004460168749
fval: -556.7535024275659
fval: -556.6815294806427
fval: -556.9068577477553
fval: -556.802662935267
fval: -556.5868864255687
fval: -556.5142170091311
fval: -556.6133101581855
fval: -556.7879772977361
fval: -556.541287810638
fval:

# EM
In order to use EM algorithm numerically, one has to be careful about tolerance. If the tolerence parameters are too small, it's slow to finish one iteration. On the other hand, due to the iterative nature of EM algorithm, if one set the tolerance too large, it will fail to converge later on. Therefore, EM is better for cold starting a optimizer. 

In [8]:
# Fit data using EM:
theta_init = np.random.rand(len(theta))
model.set_solver(my_solver, xtol_rel=1e-5, ftol_rel=1e-5, verbose_opt=True) 
model.fit(df_train, theta_init, y_col=y_col, x_col=x_col, EM_threshold=0.005, method='EM')
theta_EM = model.theta_opt

fval: -784.5256991758029
fval: -754.900354482684
fval: -739.7340489933079
fval: -731.6770682335448
fval: -727.0867949687671
fval: -724.8417032572006
fval: -724.7266406838334
fval: -724.3558081253666
fval: -723.9990313647636
fval: -723.65572771383
fval: -723.32531368526
fval: -723.007213423163
fval: -722.7008648544398
fval: -722.4057239521069
fval: -722.1212675229888
fval: -721.8469948723018
fval: -721.5824286274612
fval: -721.327114942055
fval: -721.0806232512818
fval: -720.8425457107776
fval: -720.6124964197877
fval: -720.3901105051968
fval: -720.175043123842
fval: -719.9669684255359
fval: -719.7656067301962
fval: -719.5707311426008
fval: -719.3820251900154
fval: -719.199217170325
fval: -719.0220512875981
fval: -718.8502864920604
fval: -718.6836954160103
fval: -718.5220633978516
fval: -718.3651875868393
fval: -718.2128761216186
fval: -718.0649473761493
fval: -717.9212292670591
fval: -717.7815586169919
fval: -717.6458220754134
fval: -718.5589550268119
fval: -718.4229738745073
fval: -71

In [9]:
# Make predictions from LLY:
df_LLY = model.predict(df_test, theta=theta_LLY)

# Make predictions from mixed models:
df_mix = model.predict(df_test, theta=theta_mix)

# Make predictions from EM:
df_EM = model.predict(df_test, theta=theta_EM)

# Make predictions using true theta:
df_true = model.predict(df_test, theta=theta)

# Compare Performance

In [10]:
# Calculate Statistics
RMSE = {}
RMSE['true'] = np.sqrt((df_true.y_0_filtered - df_true.y_0_vis).var())
RMSE['LLY'] = np.sqrt((df_LLY.y_0_filtered - df_LLY.y_0_vis).var())
RMSE['EM'] = np.sqrt((df_EM.y_0_filtered - df_EM.y_0_vis).var())
RMSE['mix'] = np.sqrt((df_mix.y_0_filtered - df_mix.y_0_vis).var())

M_error = {}
M_error['true'] = (df_true.y_0_filtered - df_true.y_0_vis).mean()
M_error['LLY'] = (df_LLY.y_0_filtered - df_LLY.y_0_vis).mean()
M_error['EM'] = (df_EM.y_0_filtered - df_EM.y_0_vis).mean()
M_error['mix'] = (df_mix.y_0_filtered - df_mix.y_0_vis).mean()
print(RMSE)
print(M_error)

{'true': 1.817851207863637, 'LLY': 1.818411159327936, 'EM': 1.8191099429083135, 'mix': 1.818411205491564}
{'true': 0.08827873963664018, 'LLY': -0.050676217516590696, 'EM': -0.01241504069078562, 'mix': -0.05067130414756569}
